# Testing Calibration V2Worksheet

## 1. Setup
This section load python objects and variables please check the ```cam``` variable

### 1.1 Load python objects
Loads ```scope``` and ```lamp``` python objects for reading sekonic and communication with the lamp

In [57]:
cam = "http://192.168.2.224/video_feed"
sharedFolder = "C:\\Users\\Virtual\ Production\ 2\\Documents\\ApolloLogs\\"
%run calibration_setup_v2.ipynb

### 1.2 Connect to lamp
Lamp needs to be closed before it can be opened again.
If you dont know the serial port where the lamp is connected use the Find COM port section
1st You need to setup sekonic and feedback camera (the areas where the data will be read, 1st -green value is set up so it would be possibel to get dUV - sign location), then close the lamp and start again with the scope objects

In [317]:
lamp1.close()

In [58]:
lamp1 = apollo_device(ComPort='COM6')
lamp1.getId()
lamp1.macId = lamp1.macId.replace(":","")
lamp1.macId

'30C6F7C8DD10'

In [59]:
lamp1.color = [200,50,50,500,0]
lamp1.update()

In [201]:
cmd = "I 1 1 1 255 0\n"
lamp1.send(cmd.encode('utf-8'))
lamp1.read()
print(lamp1.lastValAsJsonString)

{"mixed_red":4,"mixed_green":4,"mixed_blue":4,"mixed_white":16,"red_actual":2,"lamp_temp":24.250000,"lamp_target":65.000000,"fan_speed":0,"Timestamp":1699976598935211200}


#### [Optional] Find COM port

In [9]:
# Get COM port list
import serial.tools.list_ports

com_ports = list(serial.tools.list_ports.comports())

for port, desc, hwid in com_ports:
    print(str(port)+" "+str(desc))

COM5 USB Serial Port (COM5)
COM6 USB Serial Port (COM6)


### 1.3 Set Sekonic camera areas
Set the area where to "read" the Tcp, dUV and lux variables
Once the object is initalized tune the paramters and test by ```update()```
To save the paramteres use the Setup below to generate the ```saved_string``` and copy and paste it into funtion paramter and save the workbook

In [60]:
Brightness = scope(saved_string="self.slider_h.value = (232, 391)\nself.slider_v.value = (319, 377)\nself.minus_comp.value = False\nself.minus_brightness.value = '150'\nself.offset_x.value = 0\nself.offset_y.value = 0\nself.rota.value = '90'\nself.flip.value = False\nself.flop.value = False\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.')]) \n")

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Label(value='Out:')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x87\x00\x00\x002\x08\x02\x00\x00\x00\xf9\x82_\x9…

Output()

In [33]:
Brightness.update()

4190

In [61]:
dUV = scope(saved_string="self.slider_h.value = (258, 447)\nself.slider_v.value = (237, 295)\nself.minus_comp.value = True\nself.minus_brightness.value = '150'\nself.offset_x.value = -85\nself.offset_y.value = 3\nself.rota.value = '90'\nself.flip.value = False\nself.flop.value = False\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.'), ('20.0', '0,0'), ('0.6', '0.0'), ('0.8', '0.0')]) \n")

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Label(value='Out:')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x87\x00\x00\x002\x08\x02\x00\x00\x00\xf9\x82_\x9…

Output()

In [34]:
dUV.update()

-0.029

In [62]:
Kelvin = scope(saved_string="self.slider_h.value = (259, 417)\nself.slider_v.value = (159, 217)\nself.minus_comp.value = False\nself.minus_brightness.value = '150'\nself.offset_x.value = 0\nself.offset_y.value = 0\nself.rota.value = '90'\nself.flip.value = False\nself.flop.value = False\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.'), ('G', '6')]) \n")

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Label(value='Out:')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x87\x00\x00\x002\x08\x02\x00\x00\x00\xf9\x82_\x9…

Output()

In [36]:
Kelvin.update()

5222

#### Setup

In [49]:
Brightness.save_string()

"self.slider_h.value = (232, 391)\nself.slider_v.value = (319, 377)\nself.minus_comp.value = False\nself.minus_brightness.value = '150'\nself.offset_x.value = 0\nself.offset_y.value = 0\nself.rota.value = '90'\nself.flip.value = False\nself.flop.value = False\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.')]) \n"

In [50]:
dUV.save_string()

"self.slider_h.value = (258, 447)\nself.slider_v.value = (237, 295)\nself.minus_comp.value = True\nself.minus_brightness.value = '150'\nself.offset_x.value = -85\nself.offset_y.value = 3\nself.rota.value = '90'\nself.flip.value = False\nself.flop.value = False\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.'), ('20.0', '0,0'), ('0.6', '0.0'), ('0.8', '0.0')]) \n"

In [51]:
Kelvin.save_string()

"self.slider_h.value = (259, 417)\nself.slider_v.value = (159, 217)\nself.minus_comp.value = False\nself.minus_brightness.value = '150'\nself.offset_x.value = 0\nself.offset_y.value = 0\nself.rota.value = '90'\nself.flip.value = False\nself.flop.value = False\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.'), ('G', '6')]) \n"

# Test

In [89]:
lamp1.close()
time.sleep(0.5)

In [64]:
lamp1 = apollo_device(ComPort='COM6',SekBrightnes=Brightness, SekDuv=dUV, SekKelvin=Kelvin)
lamp1.getId()
lamp1.macId = lamp1.macId.replace(":","")
lamp1.macId

'30C6F7C8DD10'

In [87]:
%%time

startCalib = 0
endCalib = 5
readCalibration = True
writeCalibration = True

data = None

def calRange(KelvinTemp = 2, BrightnessV = 8, readCalibration = True, writeCalibration = True):
    global calibration_points
    if readCalibration:
        print("Loading calibartion from lamp\n")
        lamp1.send(("Cr"+str(KelvinTemp)+" "+str(BrightnessV)+" \n").encode('utf-8'))
        lamp1.read()
        jsonString = "{"+lamp1.lastValAsJsonString[1:-1]+"}"
        jsonObject = eval(jsonString)

        print("loaded values: "+str(jsonObject)+"\n")
        calibration_points[KelvinTemp][8][2] = jsonObject['r']
        calibration_points[KelvinTemp][8][3] = jsonObject['g']
        calibration_points[KelvinTemp][8][4] = jsonObject['b']
        calibration_points[KelvinTemp][8][5] = jsonObject['w']

        # The answer is 2 line, so to clear the buffer
        lamp1.read()

    calibration_points = lamp1.AdjustLux(BrightnessV,KelvinTemp, calibration_points)
    calibration_points = lamp1.AdjustDuv(BrightnessV,KelvinTemp, calibration_points)
    calibration_points = lamp1.AdjustKelvin(BrightnessV,KelvinTemp, calibration_points)

    #write calibration to lamp
    if writeCalibration:
        print("Writing calibratrion:\n")
        cmd = "Cw"+str(KelvinTemp)+" "+str(BrightnessV)+" "+str(calibration_points[KelvinTemp][BrightnessV][2])+" "+str(calibration_points[KelvinTemp][BrightnessV][3])+" "+str(calibration_points[KelvinTemp][BrightnessV][4])+" "+str(calibration_points[KelvinTemp][BrightnessV][5])+"\n"
        lamp1.send(cmd.encode('utf-8'))
        print(cmd)
        lamp1.read()
        print(lamp1.lastValAsJsonString)

    print(f'Temperature: {lamp1.SekKelvin.update()}K\ndUv: {lamp1.SekDuv.update()}\nBrightness: {lamp1.SekBrightnes.update()}lx')
    return [lamp1.SekKelvin.lastVal, lamp1.SekDuv.lastVal, lamp1.SekBrightnes.lastVal]

# try 5 times to get the dUV low enough
for kelvin in range(startCalib,endCalib+1):
    print(f"starting kelvin range {KelvinRangeIds[kelvin]}\n")
    done = False
    for i in range(5):
        if done:
            break  # Exit the inner loop when calibration is successful
        measured = calRange(KelvinTemp=kelvin, readCalibration=readCalibration, writeCalibration=writeCalibration)
        if abs(measured[1]) < 0.0002:
            done = True
    if done:
        # Record final results for proofing
        new_data_line = {
            'wb': [KelvinRangeIds[kelvin]],
            'brightness_point': [8],
            'kelvin': [measured[0]],
            'duv': [measured[1]],
            'lux': [measured[2]]
        }
        new_data_line_df = pd.DataFrame(new_data_line)
        print("Debug new_data_line: " + str(new_data_line))
        if data is None:
            data = new_data_line_df
        else:
            data = pd.concat([data, new_data_line_df], ignore_index=True)
        

starting kelvin range 2800

Loading calibartion from lamp

loaded values: {'wb': 0, 'brightness': 8, 'r': 1316, 'g': 660, 'b': 9, 'w': 370, 'Timestamp': 1700461623332706800}

Adjusting lamp brightness to 10000lx in 2800K with [4, 2, 0, 1]/[36, 18, 0, 10] steps
current brightness: 9830, ideal 10000, diff 170 : [1316, 660, 9, 370, 0] / [837, 660, 9, 370, 0]
current brightness: 9850, ideal 10000, diff 150 : [1320, 662, 9, 371, 0] / [840, 662, 9, 371, 0]
current brightness: 9900, ideal 10000, diff 100 : [1324, 664, 9, 372, 0] / [843, 664, 9, 372, 0]
current brightness: 9940, ideal 10000, diff 60 : [1328, 666, 9, 373, 0] / [846, 666, 9, 373, 0]
current brightness: 9980, ideal 10000, diff 20 : [1332, 668, 9, 374, 0] / [852, 668, 9, 374, 0]
current brightness: 10000, ideal 10000, diff 0 : [1336, 670, 9, 375, 0] / [855, 670, 9, 375, 0]
selected: [2800, 8, 1336, 670, 9, 375]
Adjusting lamp dUV to +/- 0.003 in 2800K with [0, 1, 0, 0]/[0, 30, 0, 0] steps
current dUV: 0.0005, ideal 0.0000, diff 0.

In [76]:
data

,wb,brightness_point,kelvin,duv,lux
0,2800,8,2811,0.0000,10000
1,3200,8,3318,-0.0001,9950
2,4800,8,4913,-0.0001,19700
3,5600,8,5603,0.0000,19800
4,7800,8,7812,0.0001,19900
5,10000,8,10018,0.0000,20000


In [81]:
%%time

startCalib = 0
endCalib = 5
readCalibration = True
writeCalibration = True


def calKelvinRange(KelvinTemp = 2, readCalibration = True, writeCalibration = True):
    global calibration_points, data

    if readCalibration:
        print("Loading calibartion from lamp\n")
        lamp1.send(("Cr"+str(KelvinTemp)+" 8 \n").encode('utf-8'))
        lamp1.read()
        jsonString = "{"+lamp1.lastValAsJsonString[1:-1]+"}"
        jsonObject = eval(jsonString)

        print("loaded values: "+str(jsonObject)+"\n")
        calibration_points[KelvinTemp][8][2] = jsonObject['r']
        calibration_points[KelvinTemp][8][3] = jsonObject['g']
        calibration_points[KelvinTemp][8][4] = jsonObject['b']
        calibration_points[KelvinTemp][8][5] = jsonObject['w']

        # The answer is 2 line, so to clear the buffer
        lamp1.read()

    for lx in reversed(range(1,8)):
        calibration_points[KelvinTemp][lx] = [round(x/2) for x in calibration_points[KelvinTemp][lx+1]]
        measured = calRange(KelvinTemp=KelvinTemp,BrightnessV=lx,readCalibration=False,writeCalibration=writeCalibration)
        # record final results for proofing
        newDataLine = {
            'wb': [KelvinRangeIds[KelvinTemp]],
            'brightness_point': [lx],
            'kelvin': [measured[0]],
            'duv': [measured[1]],
            'lux': [measured[2]]
        }
        newDataLineDf = pd.DataFrame(newDataLine)
        print("Debug newDataLine: "+str(newDataLine))
        if data is None:
            data = newDataLineDf
        else:
            data = pd.concat([data, newDataLineDf], ignore_index=True)
        
# 
for kelvin in range(startCalib,endCalib+1):
    print(f"starting kelvin range {KelvinRangeIds[kelvin]}\n")
    calKelvinRange(KelvinTemp=kelvin,readCalibration=readCalibration,writeCalibration=writeCalibration)
    

starting kelvin range 2800

Loading calibartion from lamp

loaded values: {'wb': 0, 'brightness': 8, 'r': 1316, 'g': 660, 'b': 9, 'w': 370, 'Timestamp': 1700441662820414400}

Adjusting lamp brightness to 5000lx in 2800K with [4, 2, 0, 1]/[36, 18, 0, 10] steps
current brightness: 4850, ideal 5000, diff 150 : [658, 330, 4, 185, 0] / [420, 330, 4, 185, 0]
current brightness: 4840, ideal 5000, diff 160 : [662, 332, 4, 186, 0] / [423, 332, 4, 186, 0]
current brightness: 4860, ideal 5000, diff 140 : [666, 334, 4, 187, 0] / [425, 334, 4, 187, 0]
current brightness: 4900, ideal 5000, diff 100 : [670, 336, 4, 188, 0] / [429, 336, 4, 188, 0]
current brightness: 4930, ideal 5000, diff 70 : [674, 338, 4, 189, 0] / [431, 338, 4, 189, 0]
current brightness: 4970, ideal 5000, diff 30 : [678, 340, 4, 190, 0] / [434, 340, 4, 190, 0]
current brightness: 4990, ideal 5000, diff 10 : [682, 342, 4, 191, 0] / [437, 342, 4, 191, 0]
selected: [1400, 4, 682, 342, 4, 191]
Adjusting lamp dUV to +/- 0.003 in 2800K

In [84]:
from IPython.display import display

# Get current display settings
display(pd.get_option('display.max_rows'))
display(pd.get_option('display.max_columns'))

None

None

In [83]:
data.head(50)

,wb,brightness_point,kelvin,duv,lux
0,2800,8,2811,0.0000,10000.0
1,3200,8,3318,-0.0001,9950.0
2,4800,8,4913,-0.0001,19700.0
3,5600,8,5603,0.0000,19800.0
4,7800,8,7812,0.0001,19900.0
...,...,...,...,...,...
45,2800,5,2770,-0.0003,1220.0
46,2800,4,2834,0.0006,610.0
47,2800,3,2765,0.0001,321.0
48,2800,2,2588,-0.0013,143.0


In [79]:
# Save Log
file_path = sharedFolder+"Apollo"+str(lamp1.macId)+"_"+datetime.datetime.now().strftime("%d%m%Y_%H%M%S")+"caliblog.csv"
df.to_csv(file_path, index=False)
print("Saved to: "+file_path)

# Save Calibration
with open(sharedFolder+"Apollo"+str(lamp1.macId)+"_"+datetime.datetime.now().strftime("%d%m%Y_%H%M%S")+"calib.py", "w") as file:
    file.write(str(calibration_points))


NameError: name 'sharedFolder' is not defined

In [ ]:
playSucess()

In [ ]:
'''
cmd = "Cr5 8\n"
lamp1.send(cmd.encode('utf-8'))
lamp1.read()
print(lamp1.lastValAsJsonString)
'''

In [ ]:
'''
cmd = "Cw5 1 100 30 30 21 \n"
lamp1.send(cmd.encode('utf-8'))
lamp1.read()
print(lamp1.lastValAsJsonString)
'''